<a href="https://colab.research.google.com/github/ngue01/formation_apprentissage_renforcement/blob/master/4_Exemple_de_MDP_en_2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Note : Juillet 2023**
Avant toute chose il faut mettre à jour setuptools qui n'est plus compatible avec la version de gym utilisé dans ce carnet

**Etape 1 :**
On commence par désinstaller setuptools : Tapez "Y" et enter pour valider la désinstallation

In [ ]:
!pip uninstall setuptools

Found existing installation: setuptools 60.0.0
Uninstalling setuptools-60.0.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/_distutils_hack/*
    /usr/local/lib/python3.10/dist-packages/distutils-precedence.pth
    /usr/local/lib/python3.10/dist-packages/pkg_resources/*
    /usr/local/lib/python3.10/dist-packages/setuptools-60.0.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/setuptools/*
Proceed (Y/n)? y
  Successfully uninstalled setuptools-60.0.0


Ensuite on installe la bonne version de setuptools :

**Après l'installation, il faut cliquer sur "RESTART RUNTIME"** puis passer à l'étape 2

In [ ]:
!pip install setuptools==60.0.0

  Using cached setuptools-60.0.0-py3-none-any.whl (952 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
cvxpy 1.3.2 requires setuptools>65.5.1, but you have setuptools 60.0.0 which is incompatible.


**Etape 2 :**
Ensuite on clone le dépot gym :

In [ ]:
!git clone -b 0.18.0 https://github.com/openai/gym.git

Cloning into 'gym'...
remote: Enumerating objects: 16503, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 16503 (delta 2), reused 2 (delta 0), pack-reused 16497
Receiving objects: 100% (16503/16503), 6.95 MiB | 20.15 MiB/s, done.
Resolving deltas: 100% (11629/11629), done.
Note: switching to '2c50315aabab8e5d25a59bb12b430a18e152f01f'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false



Puis on install gym (cela prend envitron 1 minute). N'oubliez pas de cliquer sur "RESTART RUNTIME" ensuite puis passez à l'étape 3

In [ ]:
%cd /content/gym
!pip install -v -e .

/content/gym
Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Obtaining file:///content/gym
  Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-_ylbutxv/gym.egg-info
  writing manifest file '/tmp/pip-pip-egg-info-_ylbutxv/gym.egg-info/SOURCES.txt'
  writing manifest file '/tmp/pip-pip-egg-info-_ylbutxv/gym.egg-info/SOURCES.txt'
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.1/39.1 MB 24.5 MB/s eta 0:00:00
  Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-4c1ypjve/Pillow.egg-info
  writing manifest file '/tmp/pip-pip-egg-info-4c1ypjve/Pillow.egg-info/SOURCES.txt'
  no previously-included directories found matching '.ci'
  writing manifest file '/tmp/pip-pip-egg-info-4c1ypjve/Pillow.egg-info/SOURCES.txt'
  Preparing metadata (setup.py) ... done
  Runn

# Processus de décision Markovien (Markov Decision Process - MDP)

**Etape 3**

**2. Exemple en 2D**

Prenons un autre exemple d'environnement proposé par OpenAI (Frozen Lake). Nous allons ici étudier comment s'effectuent les transitions entre les états. Dans l'environnement Frozen Lake, l'agent peut se trouver sur 16 états et effectuer 4 actions par état : Haut, Bas, Gauche, Droite.

&nbsp;
<br/>
&nbsp;
<br/>
<center><img src="https://github.com/AlexandreBourrieau/FICHIERS/blob/main/RL/Concept_RL10.png?raw=true" width="800"></center>
&nbsp;
<br/>
&nbsp;
<br/>

Dans cet environnement, il y a donc 4 trous (états 5, 7, 11 et 12) et un objectif (état 15). L'agent part de l'état 0 et son objectif est d'atteindre l'état 15 en trouvant le chemin le plus court. On assume que l'agent reçoit une récompense de +1 si il atteint l'objectif, de 0 s'il tombe dans un trou (et le jeu se termine) et dans les autres cas.

On considère qu'il y a 1/3 de chance que l'agent effectue correctement l'action choisie. Il y a donc 2/3 de chance que l'action soit défaillante. Mais attention, les autres actions possibles dans la liste des actions défaillantes sont celles qui sont les "plus proches" de l'action originale. Par exemple:

- Action choisie: "Droite":
  - 33% de chance d'aller à droite
  - 33% de chance d'aller en haut
  - 33% de chance d'aller en bas

- Action chosie "Haut":
  - 33% de chance d'aller en haut
  - 33% de chance d'aller à gauche
  - 33% de chance d'aller à droite   

...

Le programme suivant définit l'environnement Frozen Lake et affiche une vue simplifié de celui-ci.

In [5]:
import gym

# Instanciation de l'environnement
env = gym.make('FrozenLake-v1')

# Remise à zéro de l'environnement et affichage de la carte
env.reset()
env.render()

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don

Le programme suivant affiche les transitions possibles pour chaque état. Les numéro des actions sont les suivants:
 - 0: Gauche
 - 1: Bas
 - 2: Droite
 - 3: Haut

In [6]:
# Récupération de la table des transitions
P = env.env.P

# Affichage des valeurs contenues dans la table
for key, value in P.items():
    print (key, ": ", value)

0 :  {0: [(0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 4, 0.0, False)], 1: [(0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 4, 0.0, False), (0.3333333333333333, 1, 0.0, False)], 2: [(0.3333333333333333, 4, 0.0, False), (0.3333333333333333, 1, 0.0, False), (0.3333333333333333, 0, 0.0, False)], 3: [(0.3333333333333333, 1, 0.0, False), (0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 0, 0.0, False)]}
1 :  {0: [(0.3333333333333333, 1, 0.0, False), (0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 5, 0.0, True)], 1: [(0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 5, 0.0, True), (0.3333333333333333, 2, 0.0, False)], 2: [(0.3333333333333333, 5, 0.0, True), (0.3333333333333333, 2, 0.0, False), (0.3333333333333333, 1, 0.0, False)], 3: [(0.3333333333333333, 2, 0.0, False), (0.3333333333333333, 1, 0.0, False), (0.3333333333333333, 0, 0.0, False)]}
2 :  {0: [(0.3333333333333333, 2, 0.0, False), (0.3333333333333

Le programme suivant affiche les transitions possibles pour l'état 10. Dans cet état, l'agent peut aller vers le haut, vers le bas, à gauche ou à droite. Mais lorsque l'agent décide d'effectuer une de ces actions, il se déplace correctement vers l'état désiré avec une probabilité de 33%.

In [7]:
for key, value in P.items():
    for k, v in value.items():
        if key == 10:
            print (key, ": ", k, " : ", v)

10 :  0  :  [(0.3333333333333333, 6, 0.0, False), (0.3333333333333333, 9, 0.0, False), (0.3333333333333333, 14, 0.0, False)]
10 :  1  :  [(0.3333333333333333, 9, 0.0, False), (0.3333333333333333, 14, 0.0, False), (0.3333333333333333, 11, 0.0, True)]
10 :  2  :  [(0.3333333333333333, 14, 0.0, False), (0.3333333333333333, 11, 0.0, True), (0.3333333333333333, 6, 0.0, False)]
10 :  3  :  [(0.3333333333333333, 11, 0.0, True), (0.3333333333333333, 6, 0.0, False), (0.3333333333333333, 9, 0.0, False)]
